Install and import packages

In [ ]:
%pip install pandas
%pip install biopython
%pip install gffutils


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [gffutils]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached scikit_learn-1.1.1-cp310-cp310-macosx_12_0_arm64.whl.metadata (10 kB)
Using cached scikit_learn-1.1.1-cp310-cp310-macosx_12_0_arm64.whl (7.7 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.2
    Uninstalling scikit-learn-1.7.2:
      Successfully uninstalled scikit-learn-1.7.2
ERROR: pip's dependency resolver does not currently take into account 

In [82]:
import pandas as pd
import gffutils
import gzip
from Bio import SeqIO
from Bio.Seq import Seq

Filtering protein-coding transcripts from the human genome annotation

In [ ]:
# Load the human genome annotation (GTF) into a pandas DataFrame
# Gencode v44 (GRCh38) reference genome

gtf_file = 'gencode.v44.basic.annotation.gtf.gz' # GTF file

columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

df = pd.read_csv(gtf_file, sep='\t', header=None, comment='#', names=columns)

print(df.head())

  seqname  source     feature  start    end score strand frame  \
0    chr1  HAVANA        gene  11869  14409     .      +     .   
1    chr1  HAVANA  transcript  11869  14409     .      +     .   
2    chr1  HAVANA        exon  11869  12227     .      +     .   
3    chr1  HAVANA        exon  12613  12721     .      +     .   
4    chr1  HAVANA        exon  13221  14409     .      +     .   

                                           attribute  
0  gene_id "ENSG00000290825.1"; gene_type "lncRNA...  
1  gene_id "ENSG00000290825.1"; transcript_id "EN...  
2  gene_id "ENSG00000290825.1"; transcript_id "EN...  
3  gene_id "ENSG00000290825.1"; transcript_id "EN...  
4  gene_id "ENSG00000290825.1"; transcript_id "EN...  


In [102]:
# Transcript filtering and annotation extraction

filtered_df = df[df['feature'] == 'transcript']
filtered_df['gene_id'] = filtered_df['attribute'].str.extract('gene_id "([^"]+)"')
filtered_df['transcript_id'] = filtered_df['attribute'].str.extract('transcript_id "([^"]+)"')
filtered_df['protein_id'] = filtered_df['attribute'].str.extract('protein_id "([^"]+)"')
filtered_df['gene_name'] = filtered_df['attribute'].str.extract('gene_name "([^"]+)"')
filtered_df = filtered_df[filtered_df['attribute'].str.contains('gene_type "protein_coding"')]

gencode_transcripts = filtered_df

/var/folders/25/sc88k95s73vbz29czqt_l4g80000gn/T/ipykernel_44594/1053399215.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['gene_id'] = filtered_df['attribute'].str.extract('gene_id "([^"]+)"')
/var/folders/25/sc88k95s73vbz29czqt_l4g80000gn/T/ipykernel_44594/1053399215.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['transcript_id'] = filtered_df['attribute'].str.extract('transcript_id "([^"]+)"')
/var/folders/25/sc88k95s73vbz29czqt_l4g80000gn/T/ipykernel_44594/1053399215.py

Map coding sequences to transcripts

In [ ]:
# Create a gffutils SQLite database from the Gencode v44 GTF annotation file
# This database stores all genomic features (genes, transcripts, exons, CDS, etc.)
# This step can take >30 minutes

database_filename = 'gencode_v44'

gffutils.create_db(gtf_file, database_filename)

OperationalError: table features already exists

In [ ]:
db = gffutils.FeatureDB(database_filename)

df = gencode_transcripts.copy()

df['cds_coordinates'] = None

for idx, row in df.iterrows():
    stable_id = row['transcript_id']
    try:
        cds = list(db.children(stable_id, order_by='+end', featuretype=['CDS']))
        start_end_cds = [[i.start, i.end] for i in cds]
    except Exception:
        start_end_cds = []
    
    df.at[idx, 'cds_coordinates'] = start_end_cds

gencode_cds_coordinates = df

,seqname,source,feature,start,end,score,strand,frame,attribute,gene_id,transcript_id,protein_id,gene_name,cds_coordinates
57,chr1,HAVANA,transcript,65419,71585,.,+,.,"gene_id ""ENSG00000186092.7""; transcript_id ""EN...",ENSG00000186092.7,ENST00000641515.2,ENSP00000493376.2,OR4F5,"[[65565, 65573], [69037, 70005]]"
234,chr1,HAVANA,transcript,450740,451678,.,-,.,"gene_id ""ENSG00000284733.2""; transcript_id ""EN...",ENSG00000284733.2,ENST00000426406.4,ENSP00000409316.1,OR4F29,"[[450743, 451678]]"
302,chr1,HAVANA,transcript,685716,686654,.,-,.,"gene_id ""ENSG00000284662.2""; transcript_id ""EN...",ENSG00000284662.2,ENST00000332831.5,ENSP00000329982.2,OR4F16,"[[685719, 686654]]"
522,chr1,HAVANA,transcript,923923,944574,.,+,.,"gene_id ""ENSG00000187634.13""; transcript_id ""E...",ENSG00000187634.13,ENST00000616016.5,ENSP00000478421.2,SAMD11,"[[924432, 924948], [925922, 926013], [930155, ..."
555,chr1,HAVANA,transcript,923923,944574,.,+,.,"gene_id ""ENSG00000187634.13""; transcript_id ""E...",ENSG00000187634.13,ENST00000618323.5,ENSP00000480678.2,SAMD11,"[[924432, 924948], [925922, 926013], [930155, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998486,chrM,ENSEMBL,transcript,10470,10766,.,+,.,"gene_id ""ENSG00000212907.2""; transcript_id ""EN...",ENSG00000212907.2,ENST00000361335.1,ENSP00000354728.1,MT-ND4L,"[[10470, 10763]]"
1998493,chrM,ENSEMBL,transcript,10760,12137,.,+,.,"gene_id ""ENSG00000198886.2""; transcript_id ""EN...",ENSG00000198886.2,ENST00000361381.2,ENSP00000354961.2,MT-ND4,"[[10760, 12137]]"
1998507,chrM,ENSEMBL,transcript,12337,14148,.,+,.,"gene_id ""ENSG00000198786.2""; transcript_id ""EN...",ENSG00000198786.2,ENST00000361567.2,ENSP00000354813.2,MT-ND5,"[[12337, 14145]]"
1998513,chrM,ENSEMBL,transcript,14149,14673,.,-,.,"gene_id ""ENSG00000198695.2""; transcript_id ""EN...",ENSG00000198695.2,ENST00000361681.2,ENSP00000354665.2,MT-ND6,"[[14149, 14673]]"


In [104]:
gencode_cds_coordinates.to_csv('gencode_cds_coordinates_hg38.csv', index=False)

Map protein sequences to coding sequences

In [ ]:
# Load the human reference genome into memory as a dictionary of SeqRecord objects

fasta_path = "GCF_000001405.26_GRCh38_genomic.fna.gz" # fasta file

with gzip.open(fasta_path, "rt") as handle: 
    genome = SeqIO.to_dict(SeqIO.parse(handle, "fasta"))

In [ ]:
# Map chromosome numbers to refseq IDs

chr_to_refseq = {
    "chr1": "NC_000001.11",
    "chr2": "NC_000002.12",
    "chr3": "NC_000003.12",
    "chr4": "NC_000004.12",
    "chr5": "NC_000005.10",
    "chr6": "NC_000006.12",
    "chr7": "NC_000007.14",
    "chr8": "NC_000008.11",
    "chr9": "NC_000009.12",
    "chr10": "NC_000010.11",
    "chr11": "NC_000011.10",
    "chr12": "NC_000012.12",
    "chr13": "NC_000013.11",
    "chr14": "NC_000014.9",
    "chr15": "NC_000015.10",
    "chr16": "NC_000016.10",
    "chr17": "NC_000017.11",
    "chr18": "NC_000018.10",
    "chr19": "NC_000019.10",
    "chr20": "NC_000020.11",
    "chr21": "NC_000021.9",
    "chr22": "NC_000022.11",
    "chrX": "NC_000023.11",
    "chrY": "NC_000024.10",
    "chrM": "NC_012920.1" 
}

gencode_cds_coordinates['refseq_id'] = gencode_cds_coordinates['seqname'].map(chr_to_refseq)

In [ ]:
# Extract and concatenate CDS segments

def get_cds_sequence(row):
    chrom = row['refseq_id']
    strand = row['strand']
    coords = row['cds_coordinates']
    
    if chrom not in genome or not coords:
        return None
    
    # Concatenate all CDS segments
    seq = ''.join(str(genome[chrom].seq[start-1:end]) for start, end in coords)
    
    # Reverse complement if on negative strand
    if strand == '-':
        seq = str(Seq(seq).reverse_complement())
    
    return seq

gencode_cds_coordinates['cds_sequence'] = gencode_cds_coordinates.apply(get_cds_sequence, axis=1)

In [ ]:
# Translate CDS sequences

def cds_to_protein(cds_seq):
    if cds_seq is None or len(cds_seq) == 0:
        return None
    seq_obj = Seq(cds_seq)
    try:
        protein = str(seq_obj.translate(table=1, cds=True))
    except Exception as e:
        protein = str(seq_obj.translate(to_stop=True))
    return protein

gencode_cds_coordinates['protein_sequence'] = gencode_cds_coordinates['cds_sequence'].apply(cds_to_protein)

/opt/homebrew/lib/python3.10/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [109]:
gencode_cds_coordinates.to_csv('gencode_cds_coordinates_with_sequences_hg38.csv', index=False)

Mapping any protein segment to genomic coodinates from coding sequences

In [119]:
def prot2gene(row, query_protein):
    """
    Given a row with 'cds_sequence', 'protein_sequence', and 'cds_coordinates',
    find the genomic CDS coordinates corresponding to a peptide subsequence.
    Returns a list of coordinate lists (one per match).
    """
    protein_seq = row['protein_sequence']
    cds_seq = row['cds_sequence']
    strand = row['strand']
    coords = row['cds_coordinates']

    if not protein_seq or not cds_seq or not coords:
        return None

    # Find all occurrences of the query peptide
    matches = []
    start_idx = 0
    while True:
        idx = protein_seq.find(query_protein, start_idx)
        if idx == -1:
            break
        matches.append(idx)
        start_idx = idx + 1

    if not matches:
        return None

    # Reverse order of CDS exons if strand is negative
    exon_coords = coords[::-1] if strand == '-' else coords

    result_coords = []
    cds_len = sum(end - start + 1 for start, end in exon_coords)

    for prot_start in matches:
        prot_end = prot_start + len(query_protein)
        nuc_start = prot_start * 3
        nuc_end = prot_end * 3

        sub_coords = []
        cum_length = 0

        for start, end in exon_coords:
            exon_len = end - start + 1
            exon_start_in_cds = cum_length
            exon_end_in_cds = cum_length + exon_len

            # Determine overlap with the peptide's CDS interval
            overlap_start = max(exon_start_in_cds, nuc_start)
            overlap_end = min(exon_end_in_cds, nuc_end)

            if overlap_start < overlap_end:
                # Offset within exon
                offset_start = overlap_start - exon_start_in_cds
                offset_end = overlap_end - exon_start_in_cds

                if strand == '+':
                    genome_start = start + offset_start
                    genome_end = start + offset_end - 1
                else:
                    # For negative strand, reverse mapping
                    genome_end = end - offset_start
                    genome_start = end - offset_end + 1

                sub_coords.append([genome_start, genome_end])

            cum_length += exon_len

        # Reorder back to genomic order
        if strand == '-':
            sub_coords = sub_coords[::-1]

        result_coords.append(sub_coords)

    return result_coords

Implementation using query sequence

In [120]:
query_protein = "DLVILLYETALLSSGFSLEDPQTHANRIYRMIKLGLGIDEDDPTADDTSAAVTEEMPPLE"

gencode_cds_coordinates["matched_cds_coords"] = gencode_cds_coordinates.apply(
    lambda row: prot2gene(row, query_protein),
    axis=1
)

matches = gencode_cds_coordinates[gencode_cds_coordinates["matched_cds_coords"].notnull()]

matches[["gene_name", "seqname", "strand", "gene_id", "transcript_id", "protein_id", "matched_cds_coords"]]

,gene_name,seqname,strand,gene_id,transcript_id,protein_id,matched_cds_coords
1387198,HSP90AA1,chr14,-,ENSG00000080824.19,ENST00000216281.13,ENSP00000216281.8,"[[[102081751, 102081821], [102082111, 10208221..."
1387224,HSP90AA1,chr14,-,ENSG00000080824.19,ENST00000334701.11,ENSP00000335153.7,"[[[102081751, 102081821], [102082111, 10208221..."


In [116]:
# Check if matched_cds_coords match query peptide

from Bio import SeqIO
from Bio.Seq import Seq
import gzip

def reconstruct_cds_from_coords(genome_seq_dict, chrom, cds_coords, strand):
    """
    genome_seq_dict: dict of chromosome sequences (SeqRecords)
    chrom: chromosome name
    cds_coords: list of [start, end] genomic coordinates
    strand: '+' or '-'
    """
    cds_seq = ""
    for start, end in cds_coords:
        # Access the .seq of SeqRecord and slice as a string
        cds_seq += str(genome_seq_dict[chrom].seq[start-1:end])
    if strand == '-':
        cds_seq = str(Seq(cds_seq).reverse_complement())
    return cds_seq


# Function to check if peptide exists in any translation frame
def peptide_in_cds_any_frame(cds_seq, query_protein):
    seq = Seq(cds_seq)
    frames = []

    # Forward frames
    for i in range(3):
        frames.append(str(seq[i:].translate(to_stop=False)))

    # Reverse complement frames
    rev_seq = seq.reverse_complement()
    for i in range(3):
        frames.append(str(rev_seq[i:].translate(to_stop=False)))

    # Check for peptide
    for f in frames:
        if query_protein in f:
            return True
    return False

# Example for the first row 
row = matches.iloc[0]
chrom = row['refseq_id']
strand = row['strand']
matched_coords_list = row['matched_cds_coords']  

# Loop over all matched segments 
found = False
for matched_coords in matched_coords_list:
    cds_seq = reconstruct_cds_from_coords(genome, chrom, matched_coords, strand)
    if peptide_in_cds_any_frame(cds_seq, query_protein):
        found = True
        break

print("Query protein found in matched CDS coordinates:", found)

Query protein found in matched CDS coordinates: True


/opt/homebrew/lib/python3.10/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
